In [151]:
import re
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [152]:
URL_1 = 'https://habr.com/ru/all/'
URL_2 = 'https://habr.com/ru/search/?target_type=posts&order_by=relevance&q=python+%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3&flow='
KEYWORDS = ['python', 'парсинг']
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
FILE_1 =  'habr.csv'
FILE_2 =  'habr1.csv'

In [153]:
def get_html(url, params=None):
    r=requests.get(url, headers=HEADERS, params=params)
    return r

In [154]:
def content(html, KEYWORDS):
    soup = BeautifulSoup(html, 'html.parser')
    article=[]
    items = soup.find_all('article', class_='post post_preview')
    
    for item in items:
        post_id = item.parent.attrs.get('id')
        if not post_id:
            continue
        post_id = int(post_id.split('_')[-1])
        prewie=item.find('div', class_='post__text')
        prewie_lower = prewie.text.lower()
        if any([desired in prewie_lower for desired in KEYWORDS]):
            article.append({
                'title': item.find('h2', class_='post__title').get_text(strip=True),
                'topic': item.find('a', class_='inline-list__item-link hub-link').get_text(strip=True),
                'date': item.find('span', class_='post__time').get_text(strip=True),
                'link': item.find('a', class_='post__title_link').get('href')   
            })
    return article

In [155]:
def pages_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    pagination1 = soup.find_all('span', class_='toggle-menu__item-link toggle-menu__item-link_pagination toggle-menu__item-link_active')
    pagination2 = soup.find_all('a', class_='toggle-menu__item-link toggle-menu__item-link_pagination')
   
    if pagination1 or pagination2:
        if int(pagination1[-1].get_text()) > int(pagination2[-1].get_text()):
            return int(pagination1[-1].get_text(strip=True))
        else:
            return int(pagination2[-1].get_text(strip=True))
    else:
        return 1

In [156]:
def save_file(items, path):
    with open(path, 'w', newline='') as file:
        writer=csv.writer(file, delimiter=';')
        writer.writerow(['Заголовок', 'Ключивые слова', 'Дата публикации', 'Ссылка'])
        for item in items:
             writer.writerow([item['title'], item['topic'], item['date'], item['link']])
    print("Файл создан!")

In [157]:
# https://habr.com/ru/all/
html=get_html(URL_1)
if html.status_code == 200:
    article=[]
    pages = 50 #pages_count(html.text)
    for p in range(1, pages+1):
        print('Парсинг страницы № ', p, ' из', pages, ' страниц')
        html=get_html(URL_1+'page'+ str(p))
        article.extend(content(html.text, KEYWORDS))
    print('Данные собраны!')
    save_file(article, FILE_1)  
else:
    print("Error") #ошибка открытия сайта

Парсинг страницы №  1  из 50  страниц
Парсинг страницы №  2  из 50  страниц
Парсинг страницы №  3  из 50  страниц
Парсинг страницы №  4  из 50  страниц
Парсинг страницы №  5  из 50  страниц
Парсинг страницы №  6  из 50  страниц
Парсинг страницы №  7  из 50  страниц
Парсинг страницы №  8  из 50  страниц
Парсинг страницы №  9  из 50  страниц
Парсинг страницы №  10  из 50  страниц
Парсинг страницы №  11  из 50  страниц
Парсинг страницы №  12  из 50  страниц
Парсинг страницы №  13  из 50  страниц
Парсинг страницы №  14  из 50  страниц
Парсинг страницы №  15  из 50  страниц
Парсинг страницы №  16  из 50  страниц
Парсинг страницы №  17  из 50  страниц
Парсинг страницы №  18  из 50  страниц
Парсинг страницы №  19  из 50  страниц
Парсинг страницы №  20  из 50  страниц
Парсинг страницы №  21  из 50  страниц
Парсинг страницы №  22  из 50  страниц
Парсинг страницы №  23  из 50  страниц
Парсинг страницы №  24  из 50  страниц
Парсинг страницы №  25  из 50  страниц
Парсинг страницы №  26  из 50  стр

In [158]:
df = pd.read_csv(FILE_1, sep = ';', encoding='1251')
df

,Заголовок,Ключивые слова,Дата публикации,Ссылка
0,Вскрытие покажет: Решаем лёгкий crackme и пише...,Информационная безопасность,вчера в 16:04,https://habr.com/ru/post/526136/
1,Побег из песочницы с Python,Блог компании OTUS. Онлайн-образование,вчера в 15:54,https://habr.com/ru/company/otus/blog/526132/
2,"Шаблонные функции в Python, которые могут выпо...",Python,1 ноября 2020 в 22:33,https://habr.com/ru/post/525998/
3,Настройка синхронизации google calendar с web ...,Python,29 октября 2020 в 18:26,https://habr.com/ru/post/525680/
4,Роль математики в машинном обучении,Блог компании OTUS. Онлайн-образование,29 октября 2020 в 16:23,https://habr.com/ru/company/otus/blog/525636/
5,"Правда о регистре символов, которую должны зна...",Программирование,29 октября 2020 в 14:14,https://habr.com/ru/post/525608/
6,Пишем бот для пазл игры на Python,Python,28 октября 2020 в 11:29,https://habr.com/ru/post/525380/
7,Пять самых сложных языков программирования на ...,Блог компании Productivity Inside,28 октября 2020 в 11:15,https://habr.com/ru/company/productivity_insid...
8,Реализуем и сравниваем оптимизаторы моделей в ...,Блог компании SkillFactory,27 октября 2020 в 18:48,https://habr.com/ru/company/skillfactory/blog/...
9,AutoVIML: Автоматизированное машинное обучение,Блог компании OTUS. Онлайн-образование,27 октября 2020 в 17:23,https://habr.com/ru/company/otus/blog/525292/


In [159]:
# https://habr.com/ru/search/?target_type=posts&order_by=relevance&q=python+%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3&flow=
html=get_html(URL_2)
if html.status_code == 200:
    article=[]
    pages = pages_count(html.text)
    for p in range(1, pages+1):
        print('Парсинг страницы № ', p, ' из', pages, ' страниц')
        html=get_html(URL_2+'page'+ str(p))
        article.extend(content(html.text, KEYWORDS))
    print('Данные собраны!')
    save_file(article, FILE_2)  
else:
    print("Error") #ошибка открытия сайта

1
8
Парсинг страницы №  1  из 8  страниц
Парсинг страницы №  2  из 8  страниц
Парсинг страницы №  3  из 8  страниц
Парсинг страницы №  4  из 8  страниц
Парсинг страницы №  5  из 8  страниц
Парсинг страницы №  6  из 8  страниц
Парсинг страницы №  7  из 8  страниц
Парсинг страницы №  8  из 8  страниц
Данные собраны!
Файл создан!


In [160]:
df = pd.read_csv(FILE_2, sep = ';', encoding='1251')
df

,Заголовок,Ключивые слова,Дата публикации,Ссылка
0,Парсингна Pуthon. Как собрать архив Голубятен,Python,14 июня 2011 в 19:24,https://habr.com/ru/post/121815/
1,Как выбрать решение дляпарсингасайтов: классиф...,Разработка под e-commerce,1 октября 2020 в 17:17,https://habr.com/ru/post/521646/
2,Как проанализировать рынок фотостудий с помощь...,Python,31 июля 2020 в 10:00,https://habr.com/ru/post/512208/
3,Веб-парсингна Ruby,Ruby,6 марта 2015 в 17:40,https://habr.com/ru/post/252379/
4,Вглубь Pyparsing: парсим единицы измерения наP...,Python,28 октября 2014 в 10:00,https://habr.com/ru/post/241670/
...,...,...,...,...
155,Регулярные выражения вPythonот простого к слож...,Спортивное программирование,26 февраля 2018 в 09:23,https://habr.com/ru/post/349860/
156,"Совмещение R иPython: зачем, когда и как?",Блог компании Open Data Science,8 ноября 2018 в 14:00,https://habr.com/ru/company/ods/blog/348260/
157,Жизненный цикл кода наPython– модель выполнени...,Блог компании OTUS. Онлайн-образование,1 марта 2019 в 15:00,https://habr.com/ru/company/otus/blog/442252/
158,"Python(x, y)",Python,4 февраля 2011 в 19:01,https://habr.com/ru/post/113065/
